# Movie_Recommendation_Sys

## 라이브러리 import 및 데이터 전처리

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
ratings = ratings[ratings['rating']>=3] #괜찮은 영화를 추천해줄 것이기 때문에 3점이하로는 버린다
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
from scipy.sparse import csr_matrix

num_title = ratings['movie_id'].nunique()
num_user = ratings['user_id'].nunique()

In [8]:
print("ratings에 있는 고유한 영화 개수 : ",num_title)
print("ratings에 있는 고유한 사용자 수 : ",num_user)

ratings에 있는 고유한 영화 개수 :  3628
ratings에 있는 고유한 사용자 수 :  6039


In [9]:
data = pd.merge(ratings,movies)

In [10]:
data

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [11]:
top_30 = data.groupby('title')[['count']].sum().sort_values(by=['count'], ascending = False)

인기 많은 영화라면 많은 관람객 수와 관람객 당 평점이 높아야 한다고 생각했기 때문에 sum으로  
집계했다. 관객들이 준 평점은 낮지만 그저 사람이 많아서 높은 총 평점을 가지는 경우를 배제했다. 재미 없는 영화라면 관객도 적을 수 밖에 없을 것이라고 생각했기 때문이다.  
좀 더 신뢰성있는 데이터를 만들기 위해서는 총평점을 user_id의 count 값만큼 나눠주면 되겠다는 생각을 했다.

## 가장 인기 있는 영화 30개 

In [12]:
top_30.head(30)

,count
title,
American Beauty (1999),14449
Star Wars: Episode IV - A New Hope (1977),13178
Star Wars: Episode V - The Empire Strikes Back (1980),12648
Saving Private Ryan (1998),11348
Star Wars: Episode VI - Return of the Jedi (1983),11303
Raiders of the Lost Ark (1981),11179
"Silence of the Lambs, The (1991)",11096
"Matrix, The (1999)",10903
"Sixth Sense, The (1999)",10703


## 선호하는 영화 5가지 데이터 추가  

영화를 전체적으로 보려면 pandas의 옵션을 설정해줄 수 있다

            #user_id, movie_id, count(rating),timestamp
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)

In [13]:
movies

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [14]:
my_movie = [['012012', 356, 4,None, 'Forrest Gump (1994)', 'Comedy|Romance|War'],
            ['012012', 293, 5,None, 'Professional, The (a.k.a. Leon: The Professional) (1994)','Crime|Drama|Romance|Thriller'],
            ['012012', 1707, 5,None, 'Home Alone (1990)', 'Children\'s|Comedy'],
            ['012012', 1682, 5,None, 'Truman Show, The (1998)','Drama' ],
            ['012012', 47, 5,None, 'Seven (Se7en) (1995)', 'Crime|Thriller']]

data_size = len(data)

for i in range(len(my_movie)):
    data.loc[filtered_data_size+i] = my_movie[i]

In [15]:
data.tail(10)

,user_id,movie_id,count,timestamp,title,genre
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360,5,1016563709,Identification of a Woman (Identificazione di ...,Drama
836478,012012,356,4,None,Forrest Gump (1994),Comedy|Romance|War
836479,012012,293,5,None,"Professional, The (a.k.a. Leon: The Profession...",Crime|Drama|Romance|Thriller
836480,012012,1707,5,None,Home Alone (1990),Children's|Comedy
836481,012012,1682,5,None,"Truman Show, The (1998)",Drama
836482,012012,47,5,None,Seven (Se7en) (1995),Crime|Thriller


In [16]:
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()#중복이 없는 고유한 타이틀들이 들어가 있다.

user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}#고유한 타이틀에 번호를 매겨준다.

In [17]:
data.loc[0,['count']][0]

5

In [18]:
type(data['count'])

pandas.core.series.Series

## CSR Matrix  
많은 데이터를 응축해 하나의 배열로 나타내어 메모리를 절약할 수 있는 CSR Matrix를 사용해본다.

In [19]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6041x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836478 stored elements in Compressed Sparse Row format>

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6041 sparse matrix of type '<class 'numpy.int64'>'
	with 836478 stored elements in Compressed Sparse Column format>

## 학습

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 비슷한 영화 찾기 

비슷한 영화는 ALS에 이미 구현되어있는 similar_items()를 사용했다.

In [23]:
favorite_mov = 'Seven (Se7en) (1995)'
movie_id = title_to_idx[favorite_mov] #입력받은 영화를 인덱스로 바꿔줌(순차, 원래의 id가 아님)
similar_mov = als_model.similar_items(movie_id, N=15)#idx_to_title에 사용되는 id
similar_mov

[(220, 1.0),
 (1988, 0.8553388),
 (2855, 0.85123295),
 (3573, 0.84182),
 (3021, 0.8348631),
 (1989, 0.8325797),
 (2982, 0.82755375),
 (1336, 0.8243289),
 (2878, 0.82363605),
 (3013, 0.8231241),
 (3847, 0.81782347),
 (3694, 0.81608164),
 (1773, 0.8155686),
 (152, 0.8144753),
 (3933, 0.8137149)]

위 similar_mov는 리스트안 튜플 형태로 저장되어 있다.

### 인덱싱 오류 발생  

In [24]:
idx_to_title = {v:k for k,v in title_to_idx.items()} #v는 key, k는 value
                                                    
movie_list = []
for i in similar_mov:
    movie_list.append(idx_to_title[i[0]])


KeyError: 3847

keyError가 3694로만 떠서 한동안 애를 먹였다. 무슨 문제인지 파악할 수 가 없어서 곤란해  
하던 와중 다른 데이터를 뽑아보기로 했다.

In [ ]:
 for i in similar_mov:
    #movie_list.append(idx_to_title[i[0]])
    #idx_to_title[i[0]]
    idx = i[0]
    print(idx_to_title[2754])

위 similar_mov에 나왔던 영화 중 다른 idx를 사용했는데 영화 이름이 잘 나왔다. 

In [ ]:
idx_to_title[39] #이 곳에서는 csv파일을 참조해 movie_id가 3694인 영화의 idx를 사용했다.

In [ ]:
idx_to_title[movies.index[movies['movie_id']==3694][0]]
#movies에서 movie_id를 3694로 가지는 영화의 idx를 idx_to_title에 넣어주니 다른 영화가 나왔다.

천천히 뜯어보니, movies에서 사용되는 movie_id와 idx_to_title에서 사용되는 movie_id가 다른게 요인이라는 것을 확인했다.

In [27]:
print(len(idx_to_title))
print(len(movies))

3628
3883


데이터가 250여개만큼 차이난다..

어디서 어떻게 잘못되었는지는 파악돼서 데이터를 가공하기로했다.

In [62]:
cols = ['user_id', 'movie_id', 'count', 'title', 'genre']
mov_dat = data[cols]
mov_dat

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,012012,356,4,Forrest Gump (1994),Comedy|Romance|War
836479,012012,293,5,"Professional, The (a.k.a. Leon: The Profession...",Crime|Drama|Romance|Thriller
836480,012012,1707,5,Home Alone (1990),Children's|Comedy
836481,012012,1682,5,"Truman Show, The (1998)",Drama


In [63]:
unique_user = mov_dat['user_id'].unique()
unique_title = mov_dat['title'].unique()#중복이 없는 고유한 타이틀들이 들어가 있다.
user_to_idx = {v:k for k,v in enumerate(unique_user)}
movie_to_idx = {v:k for k,v in enumerate(unique_title)}

In [72]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = mov_dat['user_id'].nunique()
num_movie = mov_dat['movie_id'].nunique()

csr_data = csr_matrix((mov_dat['count'], (mov_dat.user_id, mov_dat.movie_id)),shape = (num_user, num_movie))
#csr_data
#pd.DataFrame(mov_dat.user_id).tail(30)


ValueError: column index exceeds matrix dimensions

CSR 매트릭스의 내부적인 기능에 대해서 완벽히 이해못했는지, 바꾸는 족족이 column exceeded error에 row exceeded error 번갈아가면서 떴다.  
위의 모델로 학습, 추천된 영화들은 인덱스로 미루어 보았을 때 의미있는 추천을 해 준 것으로 생각이되나, movie_idx와 title간의 매핑 실패는 여전히 치명적이다.  
다른 일정 때문에 잠시 미뤄둬야하겠으나, 이는 꼭 다시 고민해봐야하는 문제다.

## 유저 추천 

In [74]:
user = user_to_idx['012012']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(910, 1.2914267),
 (3061, 1.1124867),
 (898, 0.9144354),
 (3675, 0.8892703),
 (1035, 0.87223506),
 (594, 0.85470706),
 (1267, 0.83458716),
 (1247, 0.78571653),
 (928, 0.7774922),
 (908, 0.77201533),
 (541, 0.76767075),
 (1196, 0.7629175),
 (905, 0.7585552),
 (2941, 0.751809),
 (3606, 0.7456708),
 (3671, 0.7392144),
 (1244, 0.7344663),
 (1032, 0.7303821),
 (2078, 0.71569955),
 (3435, 0.71346104)]

위에서 말한 문제가 동일하게, 더 심하게 일어나서 제목으로 매핑하는 과정은 삭제했다.

## 기여도 확인 

In [80]:
movie = 3435
explain = als_model.explain(user, csr_data, itemid=movie)

[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('What Happened Was... (1994)', 0.10877167119106942),
 ('It Happened One Night (1934)', 0.09160221445067908),
 ('All Dogs Go to Heaven (1989)', 0.08390103753702027),
 ('Legend (1985)', 0.08297932915768483),
 ('Fly, The (1986)', 0.07641030797998238),
 ('Any Given Sunday (1999)', 0.07135071659892059),
 ('Blame It on Rio (1984)', 0.053084579197509155),
 ('Tin Cup (1996)', 0.05203074658737639),
 ('Lord of the Rings, The (1978)', 0.044168014958999934),
 ('Varsity Blues (1999)', 0.03648276999547674)]

추천해준 인덱스(영화)를 통해서 기여도를 확인했다. 